In [2]:
import requests, bs4
import pandas as pd
from lxml import html #html.lxml 로 써진 것. 즉 html.lxml 로 써야하는 명령어를 lxml만써도 작동하게해줌.
from urllib.request import Request, urlopen
from urllib.parse import urlencode, quote_plus, unquote
from pprint import pprint
import numpy as np

In [3]:
# 최종정리코드.
# 파일불러오기
df = pd.read_excel("rawdata/aptexcel202010.xlsx" )
df1 = df.copy()
list = []

#df2.values[14]  column 이름 위치확인
list =  df1.values[14]
list
# column이름 변경 
df1.columns = list
df1.sample()


# 불필요한내용 삭제  (다만 인덱싱을 다시 0부터 시작하게 못하긴했음.)
df1 = df1.drop(df1.index[0:15])
df1.head()



,시군구,번지,본번,부번,단지명,전용면적(㎡),계약년월,계약일,거래금액(만원),층,건축년도,도로명,해제사유발생일
15,강원도 강릉시 견소동,202,202,0,송정한신,84.945,202010,15,17500,8,1997,경강로2539번길 8,NaN
16,강원도 강릉시 견소동,202,202,0,송정한신,59.8,202010,22,15000,7,1997,경강로2539번길 8,NaN
17,강원도 강릉시 견소동,202,202,0,송정한신,84.945,202010,27,16800,12,1997,경강로2539번길 8,NaN
18,강원도 강릉시 견소동,289,289,0,송정해변신도브래뉴아파트,84.99,202010,6,20900,1,2005,경강로2539번길 22,NaN
19,강원도 강릉시 견소동,289,289,0,송정해변신도브래뉴아파트,59.99,202010,9,18300,5,2005,경강로2539번길 23,NaN
